In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import missingno
from collections import Counter

In [ ]:
!pip install catboost

In [ ]:
#machine learning models
from sklearn.linear_model import LinearRegression,Perceptron
from sklearn.svm import SVR,LinearSVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split

#model evaluation
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score

#hyperparameter tuning
from sklearn.model_selection import GridSearchCV

In [ ]:
df=pd.read_csv('/content/train.csv')

In [ ]:
df.head(10)

In [ ]:
df.isna().sum()

In [ ]:
df.info()

In [ ]:
df["extras"].unique()

In [ ]:
df[df["extras"] == "extras"]

In [ ]:
df.drop(2579, inplace=True)

In [ ]:
df["extras"].unique()

In [ ]:
cols = ["result", "toss"]


for col in cols:
    df[col] = (df[col] == "TRUE").astype("int")
df[cols]

In [ ]:
dtypes = {x:"" for x in df.columns}
dtypes

In [ ]:
dtypes = {
    'year': 'int',
    'toss': 'int',
    'wickets lost': 'float',
    'fours': 'float',
    'sixes': 'float',
    'extras': 'float',
    'run rate ': 'float',
    'average innings strike rate': 'float',
    'highest score': 'int',
    'wickets taken': 'float',
    'given extras': 'float',
    'result': 'int',
    'runs scored': 'int'
}

In [ ]:
df.info()

In [ ]:
df["highest score"].describe()

In [ ]:
for col in df.columns:
  print(col)

In [ ]:
df = df.astype(dtypes)

In [ ]:
df.info()

In [ ]:
sns.pairplot(df)

In [ ]:
contigency_tbl=pd.crosstab(df["team name"],df.result)
probability_tbl=contigency_tbl/contigency_tbl.sum()
sns.heatmap(probability_tbl,annot=True,center=0.5,cmap="Greys")

In [ ]:
# Use the astype function to convert the data type
df["boundaries"]=(df["sixes"]+df["fours"])

In [ ]:
df.head()

In [ ]:
df["wickets left"]=11-df["wickets lost"]

In [ ]:
df.head()

In [ ]:
df.plot.scatter(x='wickets lost', y='runs scored', c='wickets lost')

In [ ]:
df.isnull().sum().sort_values(ascending=False)

In [ ]:
missingno.matrix(df)

In [ ]:
categorical_cols = ["team name", "opponent", "host"]
# categorical_cols = []
numeric_cols = ["year", "toss", "wickets taken", "fours", "sixes", "extras", "run rate ", # sneaky
                "average innings strike rate", "highest score", "wickets taken", "given extras", "result","boundaries","wickets left"]

In [ ]:
one_hot_encoded_data = pd.get_dummies(df, columns=categorical_cols)
print(one_hot_encoded_data)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

In [ ]:
one_hot_encoded = encoder.fit_transform(df[categorical_cols])

In [ ]:
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_cols))

In [ ]:
df_encoded = pd.concat([df, one_hot_df], axis=1)

In [ ]:
df_encoded.drop("team name", axis=1, inplace=True)
df_encoded.drop("opponent", axis=1, inplace=True)
df_encoded.drop("host", axis=1, inplace=True)

In [ ]:
df_cleaned = df_encoded.dropna(axis=1, how='all')

In [ ]:
df['boundaries'].fillna(int(df['boundaries'].mean()), inplace=True)

In [ ]:
df['wickets left'].fillna(int(df['wickets left'].mean()), inplace=True)
df['wickets taken'].fillna(int(df['wickets taken'].mean()), inplace=True)

In [ ]:
df_cleaned.dropna(subset=['runs scored'], inplace=True)

In [ ]:
train_dataset, test_dataset = train_test_split(df_cleaned, test_size=0.2, random_state=10)

In [ ]:
def X_y_split(df: pd.DataFrame):
    X = df.drop("runs scored", axis=1)
    y = df["runs scored"]
    return X, y

In [ ]:
X_train, y_train = X_y_split(train_dataset)

In [ ]:
X_train.head()

In [ ]:
X_train.info()

In [ ]:
X_test, y_test = X_y_split(test_dataset)

In [ ]:
from sklearn.impute import SimpleImputer

# Create an imputer object
imputer = SimpleImputer(strategy="mean")

# Fit the imputer on the training data
imputer.fit(X_train)

# Transform the training and test data
X_train = imputer.transform(X_train)
X_train = pd.DataFrame(X_train)
X_test = imputer.transform(X_test)
X_test = pd.DataFrame(X_test)

In [ ]:
X_train.isna().sum()

In [ ]:
df_cleaned.reset_index(drop=True, inplace=True)

In [ ]:
svr = SVR()
svr.fit(X_train, y_train)
y_test_pred = svr.predict(X_test)
print("RMSE: ", mean_squared_error(y_test, y_test_pred, squared=False))

In [ ]:
linreg=LinearRegression()
linreg.fit(X_train,y_train)
y_test_pred = linreg.predict(X_test)
print("RMSE: ", mean_squared_error(y_test, y_test_pred, squared=False))

In [ ]:
p=Perceptron()
p.fit(X_train,y_train)
y_test_pred = p.predict(X_test)
print("RMSE: ", mean_squared_error(y_test, y_test_pred, squared=False))

In [ ]:
linsvr=LinearSVR()
linsvr.fit(X_train,y_train)
Y_pred=linsvr.predict(X_test)
y_test_pred = linsvr.predict(X_test)
print("RMSE: ", mean_squared_error(y_test, y_test_pred, squared=False))

In [ ]:
rfg=RandomForestRegressor()
rfg.fit(X_train,y_train)
Y_pred=rfg.predict(X_test)
y_test_pred = rfg.predict(X_test)
print("RMSE: ", mean_squared_error(y_test, y_test_pred, squared=False))

In [ ]:
cbr=CatBoostRegressor()
cbr.fit(X_train,y_train)
Y_pred=cbr.predict(X_test)
y_test_pred = cbr.predict(X_test)
print("RMSE: ", mean_squared_error(y_test, y_test_pred, squared=False))

In [ ]:
df_cleaned

In [ ]:
X, y = X_y_split(df_cleaned)
svr.fit(X_train, y_train)

In [ ]:
df_sub = pd.read_csv("/content/test.csv")
df_sub

In [ ]:
df_sub.info()

In [ ]:
cols = ["result", "toss"]


for col in cols:
    df_sub[col] = (df_sub[col] == "TRUE").astype("int")
df_sub[cols]

In [ ]:
df_sub["boundaries"]=(df_sub["sixes"]+df_sub["fours"])

In [ ]:
df_sub["wickets left"]=11-df_sub["wickets lost"]

In [ ]:
df_sub.head()

In [ ]:
one_hot_encoded = encoder.fit_transform(df_sub[categorical_cols])

In [ ]:
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_cols))

In [ ]:
df_sub_encoded = pd.concat([df_sub, one_hot_df], axis=1)

In [ ]:
df_sub_encoded.info()

In [ ]:
columns_df_sub=set(df_sub.columns)
columns_df=set(df_cleaned.columns)
columns_we_need=columns_df-columns_df_sub
columns_sus=columns_df_sub-columns_df

In [ ]:
columns_we_need

In [ ]:
columns_to_zero=['host_BR','host_CA','host_DE','host_ES','host_FR','host_IT','host_JP','host_US']
df_sub_encoded[columns_to_zero]=0

In [ ]:
columns_sus

In [ ]:
df_sub_encoded.drop("team name", axis=1, inplace=True)
df_sub_encoded.drop("opponent", axis=1, inplace=True)
df_sub_encoded.drop("host", axis=1, inplace=True)

In [ ]:
df_sub.info()

In [ ]:
from sklearn.impute import SimpleImputer

# Create an imputer object
imputer = SimpleImputer(strategy="mean")

# Fit the imputer on the training data
imputer.fit(df_sub_encoded)

# Transform the training and test data
df_sub = imputer.transform(df_sub_encoded)
df_sub = pd.DataFrame(df_sub)

In [ ]:
df_sub.head()

In [ ]:
columns_we_need

In [ ]:
columns_sus

In [ ]:
y_pred = pd.DataFrame()
mp = svr.predict(df_sub)
y_pred['Runs Scored'] = mp
y_pred['Id'] = y_pred.index
y_pred.set_index('Id',inplace=True)

y_pred.to_csv('submission.csv')
y_pred